# Notebook 

In [9]:
import pandas as pd
from scipy.signal import medfilt
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt

In [2]:
# Loading data
df_weiss = pd.read_csv('../data/fuellstandsensoren-glassammelstellen-weissglas.csv',sep=';')
df_gruen = pd.read_csv('../data/fuellstandsensoren-glassammelstellen-gruenglas.csv',sep=';')
df_braun = pd.read_csv('../data/fuellstandsensoren-glassammelstellen-braunglas.csv',sep=';')

df_weiss = df_weiss[['measured_at','data_distance','geo_point_2d','device_id']]
df_gruen = df_gruen[['measured_at','data_distance','geo_point_2d','device_id']]
df_braun = df_braun[['measured_at','data_distance','geo_point_2d','device_id']]

df_weiss['color'] = 'weiss'
df_gruen['color'] = 'gruen'
df_braun['color'] = 'braun'

df = pd.concat([df_weiss,df_braun,df_gruen])


In [3]:
# Data cleaning
df.dropna(subset=['data_distance'],inplace=True)
df = df[df['data_distance'] != 2500]
df = df[df['data_distance'] != 300]
df['datetime'] = pd.to_datetime(df['measured_at'])
df['seconds'] = df['datetime'].apply(lambda x: dt.timestamp(x))
df.sort_values(by=['datetime'],inplace=True)

/tmp/ipykernel_168866/1996673782.py:5: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['datetime'] = pd.to_datetime(df['measured_at'])


In [10]:
dev_df = pd.DataFrame(columns=['device_id','max_val','min_val','max_capacity'])
for dev_id in set(df['device_id'].values):
    estimates = medfilt(df[df['device_id'] == dev_id]['data_distance'],kernel_size=11)
    dev_df = pd.concat((dev_df,pd.DataFrame({'device_id':dev_id,'max_val':np.max(estimates),'min_val':np.min(estimates),'max_capacity':np.max(estimates)-np.min(estimates)},index=[0])),ignore_index=True)


/tmp/ipykernel_168866/4113225383.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dev_df = pd.concat((dev_df,pd.DataFrame({'device_id':dev_id,'max_val':np.max(estimates),'min_val':np.min(estimates),'max_capacity':np.max(estimates)-np.min(estimates)},index=[0])),ignore_index=True)


In [ ]:
dev_fuellraten = pd.DataFrame(columns=['device_id','datetime','fuellrate'])
for device_id in set(df['device_id'].values):
    device_df = df[df['device_id'] == device_id]
    estimates = medfilt(device_df['data_distance'].values,kernel_size=11)
    changes = estimates[2:] - estimates[:-2]
    last_change = 0
    for idx, change in enumerate(changes):
        if change > 300:
            change_time = device_df['datetime'].values[idx]
            interval = device_df.iloc[last_change:idx]
            last_change =idx
            min_val = min(interval['data_distance'])
    
            plt.axvline(x = device_df['datetime'].values[idx])